In [ ]:
pip install aspect-based-sentiment-analysis

In [ ]:
import aspect_based_sentiment_analysis as absa

nlp = absa.load()

In [1]:
import warnings
warnings.filterwarnings("ignore")
from google.colab import files
import pandas as pd
import numpy as np

# Sentiment - customer Serivce

In [ ]:
path2= '/content/drive/MyDrive/5153/meta_and_review_sentiment_aspect_delivery (1).csv'
df = pd.read_csv(path2)
df.shape

(133159, 24)

In [ ]:
df.dropna(subset=['review'], axis=0, inplace=True)
df.shape

(133159, 24)

In [ ]:
text_df = df[['review']]
text_df['Ori_index'] = text_df.index

In [ ]:
text_df.head(-5)

,review,Ori_index
0,this was a good buy great shampoo and conditio...,0
1,i have used kms in the past and always loved t...,1
2,i remember kms products being a prestige salon...,2
3,this is a wonderful shampoo doesnt make your h...,3
4,i have a very dry frizzy hair i have tried man...,4
...,...,...
133149,i use this shampoo along with another product ...,133149
133150,so effective within a few weeks after using th...,133150
133151,my hairdresser told me that there were natural...,133151
133152,i have never tried shea butter before but thou...,133152


### filter reivews which contain customer service aspect

In [ ]:
with pd.option_context ('display.max_colwidth', None):
  display(text_df[text_df.review.str.contains('customer.*care', regex=True)].review)

In [ ]:
import re
pattern = r"cs$"
text_df[text_df.review.apply(lambda x: 1 if re.match(pattern,x) else 0 )==1].review

Series([], Name: review, dtype: object)

In [ ]:
customer_aspect = ['customer service', 'service', 'servicing', 
                   'return', 'exchange', 'customer support',
                   'customer experience', 'customer care', 'customer assistance'] #no cs: customer service abbreviation

In [ ]:
text_df.loc[text_df['review'].str.contains('customer.*service|service|servicing|return|exchange|customer.*support|customer.*experience|customer.*care|customer.*assistance',regex=True), 'customer_aspect'] = 1
text_df.customer_aspect.fillna(0, inplace=True)

In [ ]:
with pd.option_context ('display.max_colwidth', None):
  display(text_df[text_df.customer_aspect==1].review.iloc[3])

'not only doest it work on frizzy hair but it has a horrible sweet scent  i should have returned these immediately but ended up having to throw these out  what a waste of '

In [ ]:
sum(text_df.customer_aspect==1)

2981

In [ ]:
customer_list = list(text_df[text_df.customer_aspect==1].index)

In [ ]:
long_text_512_idx_list = list(text_df[text_df.loc[:,'review'].str.len() >=1999].review.index)

In [ ]:
customer_index_removed = [x  for x in customer_list if x in long_text_512_idx_list]
len(customer_index_removed)

127

In [ ]:
#update customer index
#remove extremel long text which this package can not handle
customer_list =[x for x in customer_list if x not in customer_index_removed]

In [ ]:
len(customer_list)

2854

In [ ]:
for idx in customer_list: 
  return_policy, exchange,service,support = nlp(text_df.review.iloc[idx], aspects=['return','exchange','service','support'])
  text_df.loc[idx, 'neu_return_policy'] = return_policy.scores[0]
  text_df.loc[idx,'neg_return_policy'] = return_policy.scores[1]
  text_df.loc[idx,'pos_return_policy'] = return_policy.scores[2]
  text_df.loc[idx, 'neu_exchange'] = exchange.scores[0]
  text_df.loc[idx,'neg_exchange'] = exchange.scores[1]
  text_df.loc[idx,'pos_exchange'] = exchange.scores[2]
  text_df.loc[idx, 'neu_service'] = service.scores[0]
  text_df.loc[idx,'neg_service'] = service.scores[1]
  text_df.loc[idx,'pos_service'] = service.scores[2]
  text_df.loc[idx, 'neu_support'] = support.scores[0]
  text_df.loc[idx,'neg_support'] = support.scores[1]
  text_df.loc[idx,'pos_support'] = support.scores[2]


In [ ]:
text_df.shape

(133159, 15)

In [ ]:
# impute na by zero 
text_df = text_df.fillna(0)

In [ ]:
#output customer service aspect
text_df.to_csv('review_sentiment_aspect_cs.csv', index=None)
files.download('review_sentiment_aspect_cs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
path3 = '/content/drive/MyDrive/5153/review_sentiment_aspect_cs.csv'
df3 = pd.read_csv(path3)
df3.shape
#feel like return may be a more accuracte indicator than exchange 

(133159, 15)

In [ ]:
df3.columns

Index(['review', 'Ori_index', 'customer_aspect', 'neu_return_policy',
       'neg_return_policy', 'pos_return_policy', 'neu_exchange',
       'neg_exchange', 'pos_exchange', 'neu_service', 'neg_service',
       'pos_service', 'neu_support', 'neg_support', 'pos_support'],
      dtype='object')

In [ ]:
df3[df3.pos_service!=0][['pos_return_policy', 'pos_exchange', 'pos_service', 'pos_support']][:20]

,pos_return_policy,pos_exchange,pos_service,pos_support
16,0.397591,0.233870,0.511892,0.318874
72,0.973744,0.025994,0.145551,0.091163
192,0.004839,0.014405,0.025652,0.024588
269,0.913371,0.024039,0.149899,0.272333
368,0.006916,0.011557,0.016231,0.016808
535,0.028421,0.013778,0.116843,0.036299
541,0.998307,0.996613,0.998622,0.998055
758,0.009671,0.011631,0.010364,0.020767
901,0.160603,0.426678,0.882344,0.791063
918,0.034597,0.035517,0.017400,0.093145


In [ ]:
#df3.loc[72,'review'] #return yes
#df3.loc[269, 'review'] #should have returned it, both wrong
#df3.loc[133081,'review'] #nothing related
#df3.loc[541,'review] #correct
df3.loc[1358,'review']


'i wasnt quite sure what to expect from a nail polish that contained none of the awful chemicals found in most nail polishes  i absolutely love the conditioner  i followed all directions took of prior polish buffed my nails rinsed and dried thoroughly before applying  i had read prior reviews about not letting your nails get wet immediately after so i polished at night  i followed these instructions however i did not read anything about this rule on the conditioner container  the polish dries very quickly and after a week of working in the yard and in the kitchen i had no chips  they had dulled a little but the polish did not come off  it was a little tricky to get off though  i had read bad reviews on the acquarella nail polish remover so i purchased a different more natural brand and unfortunately that hasnt arrived yet in the mail yet  i had to use the bad stuff acetone and it took a bit of time to get it off  however the little bit of work to get the polish off was totally worth ho

In [ ]:
df3.columns

Index(['review', 'Ori_index', 'customer_aspect', 'neu_return_policy',
       'neg_return_policy', 'pos_return_policy', 'neu_exchange',
       'neg_exchange', 'pos_exchange', 'neu_service', 'neg_service',
       'pos_service', 'neu_support', 'neg_support', 'pos_support'],
      dtype='object')

In [ ]:
df.columns

Index(['Unnamed: 0', 'asin', 'review', 'overall', 'helpful', 'summary',
       'unixReviewTime', 'reviewTime', 'reviewerID', 'title', 'brand', 'price',
       'salesRank', 'polarity_score', 'neutral_score', 'negative_score',
       'positive_score', 'neu_delivery', 'neg_delivery', 'pos_delivery',
       'neu_shipment', 'neg_shipment', 'pos_shipment', 'review_len'],
      dtype='object')

In [ ]:
#merge customer service aspect, with delivery aspect and overall sentiment score 
df_full = df.merge(df3, how = 'left', left_index=True, right_on= 'Ori_index')

In [ ]:
df_full.shape

(133159, 39)

In [ ]:
df_full.to_csv('meta_and_review_sentiment_aspect_delivery_cs.csv', index=None)

In [ ]:
files.download('meta_and_review_sentiment_aspect_delivery_cs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>